In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [2]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             #load_in_8bit=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto"
                                            )

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [11]:
import pandas as pd

data_train_tw = pd.read_csv('weibo_twitter_2024_text.csv', encoding = 'utf-8-sig')
print(data_train_tw)

          关键词                                               连接.1  \
0       eVTOL  https://weibo.com/1659280190/NzRLNoI5S?refer_f...   
1       eVTOL  https://weibo.com/5320254069/NzRwy85RE?refer_f...   
2       eVTOL  https://weibo.com/6906214710/NzR06AnSg?refer_f...   
3       eVTOL  https://weibo.com/2245266941/NzPGsDD9m?refer_f...   
4       eVTOL  https://weibo.com/1988518703/NzPGsvWgj?refer_f...   
...       ...                                                ...   
128015   vtol  https://twitter.com/EugeneGuy14/status/1741316...   
128016   vtol  https://twitter.com/Kastro44_/status/174130862...   
128017   vtol  https://twitter.com/SmestarZ/status/1741300400...   
128018   vtol  https://twitter.com/KaleiSplatoon/status/17412...   
128019   vtol  https://twitter.com/Throwplate/status/17412491...   

                   博主名 verified_type         nation  year  \
0       QuantHedgeFund            --          china  2024   
1                 潘哥悟市            --          china  2024   


In [13]:
#data_train_tw['cleaned_text'] = data_train_tw['Title'] + data_train_tw['Abstract']
data_train_tw['连接.1']

0         https://weibo.com/1659280190/NzRLNoI5S?refer_f...
1         https://weibo.com/5320254069/NzRwy85RE?refer_f...
2         https://weibo.com/6906214710/NzR06AnSg?refer_f...
3         https://weibo.com/2245266941/NzPGsDD9m?refer_f...
4         https://weibo.com/1988518703/NzPGsvWgj?refer_f...
                                ...                        
128015    https://twitter.com/EugeneGuy14/status/1741316...
128016    https://twitter.com/Kastro44_/status/174130862...
128017    https://twitter.com/SmestarZ/status/1741300400...
128018    https://twitter.com/KaleiSplatoon/status/17412...
128019    https://twitter.com/Throwplate/status/17412491...
Name: 连接.1, Length: 128020, dtype: object

In [12]:
def generate_prompt(content):
    begin = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
    #syst = "<<SYS>> You are a helpful AI assistant that answers questions briefly and directly.\n If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n"
    #inst = "Read the following text. Does it mention the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant.\n" + content
    syst = "You are a helpful AI assistant that answers questions briefly and directly.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>"
    inst = "The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: \n1: Safety \n2: Aerodynamics \n3: Integration and Infrastructure \n4: Automation \n5: Price and cost \n6: Policy \n7: User experience \nOnly answer the attitude and category number, don't answer anything else. For example: Positive, 5\nText: "+content
    end = "<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n"
    prompt = (" ").join([begin, syst, inst, end])
    return prompt

#print(generate_prompt('How are you?'))
#print(generate_prompt(data_train['text_cleaned'][1]))
print(generate_prompt(data_train_tw['cleaned_text'][1]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful AI assistant that answers questions briefly and directly.<|eot_id|>
<|start_header_id|>user<|end_header_id|> The following texts are research or tweets about urban low-altitude transportation. Rank the attitude as positive, negative, or neutral, and categorize the discussion content into one of the following categories: 
1: Safety 
2: Aerodynamics 
3: Integration and Infrastructure 
4: Automation 
5: Price and cost 
6: Policy 
7: User experience 
Only answer the attitude and category number, don't answer anything else. For example: Positive, 5
Text: 据深圳发布，近日，“深圳市低空融合飞行示范区eVTOL首飞测试”在大梅沙滨海公交场站完成。厚为华圣公司现场展示了EH216-S型无人驾驶载人飞行器绕场飞行。该飞行器已取得中国民航局颁发的型号合格证和标准适航证，成为全球首个获得适航证的eVTOL。 <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>



In [ ]:
import pandas as pd
import numpy as np
import time

data = []
save_interval = 200

start_time = time.time()  # 记录开始时间

for i in range(len(data_train_tw)):#['text_cleaned']):
    if i % 1 == 0:
        #if pd.isna(data_train_tw.iloc[i]['result']) and not pd.isna(data_train_tw.iloc[i]['cleaned_text']):
        if not pd.isna(data_train_tw.iloc[i]['cleaned_text']):
            #print(i)
            
            input_prompt = generate_prompt(data_train_tw.iloc[i]['cleaned_text'])
            #input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
            #inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
            inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
            input_tokens = inputs["input_ids"]
            attention_mask = inputs["attention_mask"]
            with torch.cuda.amp.autocast():
                generation_output = model.generate(
                    input_ids=input_tokens,
                    attention_mask=attention_mask,
                    max_new_tokens=8,
                    do_sample=False,
                    repetition_penalty=1.1,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id
                )
            op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
            #print(op)

            inst_index = op.find('assistant\n')
        
            if inst_index != -1:
                #print(text)
                #print(op[inst_index + len('assistant\n'):])
                data.append({"number": i, "link": data_train_tw.iloc[i]['连接.1'], "text": data_train_tw.iloc[i]['cleaned_text'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
                #data.append({"number": i, "link": data_train_tw.iloc[i]['Title'], "text": data_train_tw.iloc[i]['Abstract'], "result": op[inst_index + len('assistant\n'):].replace('\n', ' ')})
            else:
                print("未找到'assistant\n'标记")
                #data.append({"number": i, "text": text, "sentiment":""})
    
            # 每 save_interval 个迭代保存一次
            if len(data) == save_interval:
                df = pd.DataFrame(data)
                df.to_csv("output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)  # 追加模式
                data = []
                end_time = time.time()  # 记录结束时间
                elapsed_time = end_time - start_time  # 计算用时
                print(f"已保存到 output.csv，当前迭代次数：{i}，用时：{elapsed_time:.2f} 秒")
        
                start_time = end_time  # 更新开始时间，用于计算下一个周期的用时

# 最后一次保存
df = pd.DataFrame(data)
df.to_csv("output_tw.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)
print("已保存到 output_tw.csv")
            

已保存到 output.csv，当前迭代次数：199，用时：93.20 秒
